# [Advent of Code 2022](https://adventofcode.com/2022)
<div align="right"><i>Ben Emery<br>December 2022</i></div>


## The toolbox

Generalised pieces of code that either can be used in multiple questions or that simply makes understand the implementation easier.

In [427]:
from collections import defaultdict
import re


def Input(day, parser=str.strip, whole_file=False):
    "Fetch the data input from disk."
    filename = f"../data/advent2022/input{day}.txt"
    with open(filename) as fin:
        if whole_file:
            return parser(fin)
        return mapt(parser, fin)


def mapt(fn, *args):
    "Do a map, and convert the results to a tuple"
    return tuple(map(fn, *args))


class Node:
    def __init__(self, name, data=None):
        self.name = name
        self.data = data
        self._children = []
        self.parent = None

    def __str__(self):
        return f"<Node name={self.name} parent={self.parent.name if self.parent else None} data={self.data}>"

    def add_child(self, node):
        self._children.append(node)
        node.parent = self

    @property
    def children(self):
        return self._children[:]

    @property
    def root(self):
        if self.parent:
            return self.parent.root
        return self


def visit_post_order(node: Node):
    for child in node.children:
        yield from visit_post_order(child)
    yield node

## [Day 1](https://adventofcode.com/2022/day/1)

Nothing too difficult for day 1, other than remembering how jupyter notebooks / python works...

### Part 1

In [428]:
data = Input(1, lambda s: int(s) if s.strip() else None)

def total_calories(data):
    elves = [0]
    for food in data:
        if food is None:
            elves.append(0)
            continue
        elves[-1] += food
    return elves

elves = total_calories(data)

max(elves)

70296

In [429]:
assert _ == 70296, "Day 1.1"

### Part 2

In [430]:
elves.sort()
sum(elves[-3:])

205381

In [431]:
assert _ == 205381, "Day 1.2"

## [Day 2](https://adventofcode.com/2022/day/2)

Again we're being eased in quite gently, unless you get dicts the wrong way around of course..

### Part 1

In [432]:
data = Input(2, lambda s: s.strip().split(" "))

ROCK = "R"
PAPER = "P"
SCISSORS = "S"

SCORES = {ROCK: 1, PAPER: 2, SCISSORS: 3}
WINNING = {SCISSORS: ROCK, ROCK: PAPER, PAPER: SCISSORS}
LOSING = dict((v, k) for k, v in WINNING.items())


def map_hands(player_1, player_2):
    map_1 = {"A": ROCK, "B": PAPER, "C": SCISSORS}
    map_2 = {"X": ROCK, "Y": PAPER, "Z": SCISSORS}
    return map_1[player_1], map_2[player_2]


def score_game(p1, p2):
    score = 3 if p1 == p2 else 0
    if p2 == WINNING[p1]:
        score = 6
    return score + SCORES[p2]


def score_all_games(data, hand_mapper):
    hands = (map_hands(*d) for d in data)
    return sum(score_game(*h) for h in hands)


score_all_games(data, map_hands)

11449

In [433]:
assert _ == 11449, "Day 2.1"

### Part 2

In [434]:
def map_hands(player_1, player_2):
    map_1 = {"A": "R", "B": "P", "C": "S"}
    p1 = map_1[player_1]

    if player_2 == "X":
        p2 = LOSING[p1]
    elif player_2 == "Y":
        p2 = p1
    elif player_2 == "Z":
        p2 = WINNING[p1]

    return p1, p2


score_all_games(data, map_hands)

13187

In [435]:
assert _ == 13187, "Day 2.2"

## [Day 3](https://adventofcode.com/2022/day/3)

First time we need a set, membership tests are O(1) so they make sense to use here. Python makes it quite easy to extend behaviour for part 2, but I find I'm missing a more functional approach..

### Part 1

In [436]:
data = Input(3)


def split_halves(contents):
    mid = int(len(contents) / 2)
    return contents[:mid], contents[mid:]


def find_common(*chunks):
    common = set(chunks[0])
    for chunk in chunks[1:]:
        common &= set(chunk)
    return tuple(common)


def score(s):
    ordinal = ord(s)
    if ordinal > 96:
        score = ordinal - ord("a") + 1
    else:
        score = ordinal - ord("A") + 26 + 1
    return score


def solve(all_data, grouper):
    total = 0
    for chunks in grouper(all_data):
        common = find_common(*chunks)
        total += score(common[0])
    return total


solve(data, lambda lines: map(split_halves, lines))

7967

In [437]:
assert _ == 7967, "Day 3.1"

In [438]:
def group_threes(lines):
    for idx in range(0, len(lines) - 1, 3):
        yield lines[idx : idx + 3]


solve(data, lambda lines: group_threes(lines))

2716

In [439]:
assert _ == 2716, "Day 3.2"

## [Day 4](https://adventofcode.com/2022/day/4)


I quite enjoyed this one, refactoring the check for detecting supersets meant that I could solve the second part with little effort, which is useually the besy way to go!

In [440]:
def parse_line(line):
    m = re.match("(\d+)-(\d+),(\d+)-(\d+)", line)
    a, b, c, d = m.groups()
    return ((int(a), int(b)), (int(c), int(d)))


data = Input(4, parse_line)


def count_supersets(lines, is_within):
    return sum(is_within(p1, p2) or is_within(p2, p1) for p1, p2 in lines)


def contains_entirely(p1, p2):
    return p1[0] >= p2[0] and p1[1] <= p2[1]


count_supersets(data, contains_entirely)

453

In [441]:
assert _ == 453, "Day 4.1"

### Part 2

In [442]:
def contains_partially(p1, p2):
    return (p1[0] >= p2[0] and p1[0] <= p2[1]) or (p1[1] <= p2[1] and p1[1] >= p2[0])


count_supersets(data, contains_partially)

919

In [443]:
assert _ == 919, "Day 4.2"

## [Day 5](https://adventofcode.com/2022/day/5)

Today's felt like one of those puzzles where the hardest part was parsing the input! Reminding myself how python's regex matches works was fun though, and it's also a good lesson about something I've struggled with in the past: pragmatism. Parsing the data doesn't need to be dynamic, we can apply what we know from the inputs to help us out (that there are 9 stacks for example). In fancier language I'd say that we can use a heuristic to simplify things, but who needs that first thing in the morning?

### Part 1

In [444]:
def parse_data(lines):
    stacks = [[], [], [], [], [], [], [], [], []]
    instructions = []

    def add_to_stack(line):
        for match in re.finditer("[A-Z]", line):
            stack = match.start() // 4
            stacks[stack].insert(0, match.group())

    def add_instruction(line):
        move, pos_from, pos_to = mapt(int, re.findall("\d+", line))
        instructions.append((move, pos_from - 1, pos_to - 1))

    stacks_complete = False
    for line in lines:
        if not line.strip():
            stacks_complete = True
            continue

        if not stacks_complete:
            add_to_stack(line)
        else:
            add_instruction(line)

    return stacks, instructions


data = Input(5, parse_data, whole_file=True)


def single_mover(stacks, instruction):
    move, pos_from, pos_to = instruction
    for _ in range(move):
        val = stacks[pos_from].pop()
        stacks[pos_to].append(val)


def follow_instructions(stacks, instructions, mover):
    new_stacks = [s[:] for s in stacks]

    for instruction in instructions:
        mover(new_stacks, instruction)
    return new_stacks


def top_crates(stacks):
    return "".join(s[-1] for s in stacks)


stacks = follow_instructions(*data, single_mover)
top_crates(stacks)

'LBLVVTVLP'

In [445]:
assert _ == "LBLVVTVLP", "Day 5.1"

### Part 2

In [446]:
def bulk_mover(stacks, instruction):
    move, pos_from, pos_to = instruction
    stacks[pos_to].extend(stacks[pos_from][-move:])
    stacks[pos_from] = stacks[pos_from][:-move]


stacks = follow_instructions(*data, bulk_mover)
top_crates(stacks)

'TPFFBDRJD'

In [447]:
assert _ == "TPFFBDRJD", "Day 5.2"

## [Day 6](https://adventofcode.com/2022/day/6)

Still nothing supuer difficult about this problem, using sets to track uniqueness may be a bit much, but given the size of the data it still returns instantly so why not!

### Part 1

In [448]:
data = Input(6)[0]


def iter_chunks(data, n):
    for i in range(0, len(data) - n):
        yield i, data[i : i + n]


def count_chars_to_packet(data, length=4):
    for idx, chunk in iter_chunks(data, length):
        if len(set(chunk)) == length:
            break
    chars_processed = idx + length
    return chars_processed


count_chars_to_packet(data, 4)

1598

In [449]:
assert _ == 1598, "Day 6.1"

### Part 2

In [450]:
count_chars_to_packet(data, 14)

2414

In [451]:
assert _ == 2414, "Day 6.2"

## [Day 7](https://adventofcode.com/2022/day/7)

First tree based problem! Had a really annoying bug with files / directories that had the same name, so ended up overwriting parts of the tree. Blugh. Significant part of the work went into building the tree, I suppose I didn't really need to do that, just build the sizes as you go and don't duplicate contributions to the same parent...

But that's waaaay less fun!


### Part 1

In [456]:
data = Input(7)


def _group_commands(lines):
    """Convert lines into a command, arguments, and outputs"""
    _, current_command, *args = lines[0].split(" ")
    outputs = []

    for line in lines[1:]:
        if line.startswith("$"):
            yield (current_command, args), outputs
            _, current_command, *args = line.split(" ")
            outputs = []
        else:
            outputs.append(tuple(line.split(" ")))
    yield (current_command, args), outputs


def build_tree(output_lines):
    """Take input data and build a directory tree"""
    node_lookup = {}

    def dir_key(parent, dir_name):
        if not parent:
            return dir_name
        return dir_key(parent.parent, parent.name) + "/" + dir_name

    def cd(args, _, current_node, node_lookup):
        new_dir = args[0]
        if new_dir == "..":
            return current_node.parent

        key = dir_key(current_node, new_dir)
        if key not in node_lookup:
            node = Node(new_dir)
            if current_node:
                current_node.add_child(node)
            node_lookup[key] = node

        return node_lookup[key]

    def ls(args, cmd_outputs, current_node, node_lookup):
        for chunks in cmd_outputs:
            if chunks[0] == "dir":
                dirname = chunks[1]
                key = dir_key(current_directory, dirname)

                if key not in node_lookup:
                    node_lookup[key] = Node(dirname)
                    current_node.add_child(node_lookup[key])
            else:
                size, filename = chunks
                current_node.add_child(Node(filename, int(size)))
        return current_node

    # execute all the commands
    current_directory = None
    for (cmd, args), outputs in _group_commands(output_lines):
        if cmd == "cd":
            command = cd
        elif cmd == "ls":
            command = ls
        else:
            raise Error(f"Unknown command {cmd}")
        current_directory = command(args, outputs, current_directory, node_lookup)
    return current_directory.root


def get_sub_directory_sizes(tree):
    sizes = defaultdict(int)

    for node in visit_post_order(tree):
        if not node.parent:
            # is root, don't care
            continue

        if node.data is not None:
            # contribute directly to the parent directory's size
            sizes[node.parent] += node.data
        else:
            # we'll have visited all the children of sub-directories before,
            # so use the cached result
            sizes[node.parent] += sizes[node]

    return sizes


def sum_directories_below_limit(data):
    tree = build_tree(data)
    sizes = get_sub_directory_sizes(tree)

    total = 0
    for dirname, size in sizes.items():
        if size <= 100000:
            total += size
    return total


sum_directories_below_limit(data)

1297683

In [453]:
assert _ == 1297683, "Day 7.1"

### Part 2

Now all the hard work of the first section is done, the second was pretty simple.

In [454]:
def find_smallest_directory_to_delete(data):
    tree = build_tree(data)
    directory_sizes = get_sub_directory_sizes(tree)

    total_size = directory_sizes[tree]

    remaining_space = 70000000 - total_size
    required_space = 30000000

    # start with the root node
    smallest = tree
    for node, size in directory_sizes.items():
        if node.data is not None:
            # is a file, skip
            continue

        if remaining_space + size > required_space:
            if size < directory_sizes[smallest]:
                smallest = node

    return directory_sizes[smallest]


find_smallest_directory_to_delete(data)

5756764

In [455]:
assert _ == 5756764, "Day 7.2"